In [26]:
import numpy as np
import gemmi
import mdtraj as md
from matplotlib import pyplot as plt

In [72]:
# the "FAD frame" is:
#  +x from C10 to C9A
#  +y from N5 to N10
#  +z completing the right hand rule

# from Pauszek et al JACS 2016, 138, 14880−14889 DOI:10.1021/jacs.6b06449 

TDM_IN_FAD_FRAME_S01 = np.array([-0.3059, 0.3008, -0.0162])
TDM_IN_FAD_FRAME_S02 = np.array([-1.1889, 0.0081, -0.1393])

CHAIN_A_FAD_N5 = np.array([14.968,  13.196,  -4.538])
CHAIN_B_FAD_N5 = np.array([ 8.418, -13.783, -26.245])

In [67]:
def crystal_to_fad(trj, chain=0):

    if chain not in [0, 2]:
        raise ValueError('You want chain = 0 (A) or 2 (B)')

    approx_basis_vectors = []

    for i, atom in enumerate(["N10", "N5", "C9A", "C10"]):

        atom_sele = f'name {atom}'
        selection = f'(chainid {chain}) and (resname FDA) and ({atom_sele})'
        sele = trj.top.select(selection)

        assert len(sele) == 1, len(sele)

        xyz = trj.xyz[0, sele, :]
        v_hat = xyz[1] - xyz[0]
        v_hat = v_hat / np.linalg.norm(v_hat)
        approx_basis_vectors.append(v_hat)

    # this is approximate, but there is no way to do better from the information I have
    #   1. we assign N10 -> N5 as +x
    #   2. the, we assign (N10 -> N5) x (C9A -> C10) as +z
    #   3. finally, (x) x (z) as +y, which should be approximately C9A -> C10

    x = approx_basis_vectors[0]
    z = np.cross(x, approx_basis_vectors[1])
    y = np.cross(z, x)

    assert np.abs(np.dot(approx_basis_vectors[1], y)) > 0.99

    R = np.column_stack([x, y, z])
    det = np.linalg.det(R)
    assert np.abs(det - 1.0) < 0.001, det

    return R


def rotation_matrix_from_A(A):
    """
    For an orthorhombic system, we can write the A matrix as:

        A = R S I

    Where:
        -- I is the identity
        -- S is a (diagonal) scaling matrix that sets the length of each a/b/c/ axis
        -- R is a rotation matrix

    So, we'll simply write:

        R = A S^{-1}
    """

    if not A.shape == (3, 3):
        raise ValueError('A not 3x3', A.shape)

    # cell lengths in nm (CrystFEL uses nm)
    S = np.diag([70.200,  117.760,  170.470]) * 0.1
    R = np.dot(A, np.linalg.inv(S))

    # rotation matrices should have det(R) = 1
    det = np.linalg.det(R)
    assert np.abs(det - 1.0) < 0.01, det

    return R


def expand_p212121(vector):

    sg = gemmi.SpaceGroup('P212121')
    ops = sg.operations()

    set_of_vectors = []
    for op in ops:
        vt = op.apply_to_xyz(vector)
        set_of_vectors.append(vt)

    return np.array(set_of_vectors)


def compute_tdm_Pz(tdm_vector_in_crystal_setting, A):
    """
    From a tdm vector, in the crystal setting, return the component perpindicular to z
      1. symmetry expand
      2. rotate
      3. project on z
      4. return Pz
    """

    tdm_hat = tdm_vector_in_crystal_setting / \
        np.linalg.norm(tdm_vector_in_crystal_setting)
    
    
    z_hat = np.array([0.0, 0.0, 1.0])

    tdm_set = expand_p212121(tdm_hat)
    assert tdm_set.shape[-1] == 3

    R = rotation_matrix_from_A(A)

    Pz = np.sum(1.0 - np.square(np.dot(np.dot(R, tdm_set), z_hat)))

    return Pz


### TDM

Logic. Given TDM $\mu$ in the FAD frame of reference, we need to compute the set of TDMs in the experimental frame. To do this, we:

1. for each chain: compute the rotation matrices to go from the crystal frame to the frame of FAD (2 rotation matrices)
2. compute the TDM for each chain in the crystal frame (2 vectors)
3. symmetry-expand these vectors (4 + 4 vectors)
4. rotate these vectors using the A-matrix into the experimental frame

Then we can simply compute the component of the TDM that is in the (circular) polarization plane as:

$$
\sum_i 1 - (\vec{\mu}_i \cdot \vec{z})^2
$$

In code, we refer to this quantity as "Pz"

NB! The A-matices were computed such that (a,b,c) formed the ROWS of the matrix, but here we transpose them such that they form the COLUMNS, which is more standard.

In [18]:
a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_dark.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_TD_all.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_l+d.npy'
 
A_matrices = np.transpose(np.load(a_matrix_file), axes=(0,2,1))
print(A_matrices.shape)

(115492, 3, 3)


In [89]:
trj = md.load_pdb('../pltest.pdb')
R = crystal_to_fad(trj, chain=0)
print(R)

tdm = np.dot(R, TDM_IN_FAD_FRAME_S02)

[[-0.69999415  0.20605138 -0.68377084]
 [-0.508037   -0.81658137  0.27400893]
 [-0.50190294  0.5391855   0.67628396]]


In [90]:
atom = CHAIN_A_FAD_N5

print(f"pseudoatom pt1, pos={[a for a in atom]}")
print(f"pseudoatom pt2, pos={[a for a in 10*tdm+atom]}")
print("distance /pt1, /pt2")
print("set dash_gap, 0")


pseudoatom pt1, pos=[14.968, 13.196, -4.538]
pseudoatom pt2, pos=[23.94823020941019, 9.929026624053716, 2.671482635259629]
distance /pt1, /pt2
set dash_gap, 0
